#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [67]:
%pip install folium
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [68]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

Загрузим данные из csv файла в датафрейм.

In [69]:
df = pd.read_csv("input_data.csv", sep=';')

Отобразим на карте координаты наших построек.

In [70]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

In [71]:
df.describe()

,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
count,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.085038e+07,7.152596e+06,1.135815e+07,8.096207e+06
mean,6.787516e+06,6.426675e+00,1.176266e+01,1.719417e+00,5.310356e+01,-2.674071e+00,5.440394e+01,5.229570e+01,1.017820e+00,5.275366e-01,4.031717e+05,3.525623e+05,5.125515e+01,1.703792e+06
std,1.977118e+08,5.283144e+00,7.218441e+00,1.157606e+00,2.712845e+01,3.241855e+01,4.582962e+00,2.183899e+01,1.562077e+00,8.813503e-01,1.959618e+05,1.366485e+05,2.250527e+01,6.404578e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e+02,4.143758e+01,-1.732189e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.161870e+05,1.000000e+00,5.886060e+05
25%,2.600000e+06,2.000000e+00,5.000000e+00,1.000000e+00,3.650000e+01,0.000000e+00,5.319411e+01,3.763847e+01,0.000000e+00,0.000000e+00,1.961580e+05,2.339150e+05,3.100000e+01,1.162465e+06
50%,3.995000e+06,5.000000e+00,1.000000e+01,2.000000e+00,4.670000e+01,6.500000e+00,5.564688e+01,4.202111e+01,0.000000e+00,0.000000e+00,4.000070e+05,3.502220e+05,5.400000e+01,1.703143e+06
75%,6.500000e+06,9.000000e+00,1.700000e+01,2.000000e+00,6.300000e+01,1.050000e+01,5.687362e+01,6.547322e+01,2.000000e+00,2.000000e+00,6.250070e+05,4.705370e+05,7.200000e+01,2.255297e+06
max,6.355524e+11,5.000000e+01,5.000000e+01,9.000000e+00,4.999000e+02,4.080000e+02,7.350889e+01,1.792583e+02,6.000000e+00,2.000000e+00,8.621630e+05,5.885970e+05,2.000000e+02,2.839173e+06


In [72]:
df.dtypes

date              object
price              int64
level              int64
levels             int64
rooms              int64
area             float64
kitchen_area     float64
geo_lat          float64
geo_lon          float64
building_type      int64
object_type        int64
postal_code      float64
street_id        float64
id_region          int64
house_id         float64
dtype: object

In [73]:
df.memory_usage()

Index                 132
date             90865200
price            90865200
level            90865200
levels           90865200
rooms            90865200
area             90865200
kitchen_area     90865200
geo_lat          90865200
geo_lon          90865200
building_type    90865200
object_type      90865200
postal_code      90865200
street_id        90865200
id_region        90865200
house_id         90865200
dtype: int64

In [74]:
df['geo_lat'] = df['geo_lat'].astype(np.float32) # чтобы памяти поменьше занимало преобразуем к более экономным типам
df['geo_lon'] = df['geo_lon'].astype(np.float32)
df['level'] = df['level'].astype(np.ubyte)
df['levels'] = df['levels'].astype(np.ubyte)
df['rooms'] = df['rooms'].astype(np.byte)
df['area'] = df['area'].astype(np.float32)
df['kitchen_area'] = df['kitchen_area'].astype(np.float32)
df['building_type'] = df['building_type'].astype(np.ubyte)

In [75]:
df.memory_usage()

Index                 132
date             90865200
price            90865200
level            11358150
levels           11358150
rooms            11358150
area             45432600
kitchen_area     45432600
geo_lat          45432600
geo_lon          45432600
building_type    11358150
object_type      90865200
postal_code      90865200
street_id        90865200
id_region        90865200
house_id         90865200
dtype: int64

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [76]:
def haversine_distance(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6378.137 * c
    return km

In [77]:
%%time
MOSCOW = (55.7558, 37.6176)  # Красная площадь
SPB = (59.9311, 30.36) # Дворцовая площадь

# признак близости к Москве и СПб
df['is_Moscow'] = df.apply(
    lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], 
                                 MOSCOW[0], MOSCOW[1]) <= 20, 
    axis=1
).astype(np.bool);

df['is_Saint_Petersburg'] = df.apply(
    lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], 
                                 SPB[0], SPB[1]) <= 20, 
    axis=1
).astype(np.bool); # все храним как бул

CPU times: user 3min 7s, sys: 3.22 s, total: 3min 11s
Wall time: 3min 13s


In [78]:
df['is_Moscow'].value_counts()

is_Moscow
False    10331278
True      1026872
Name: count, dtype: int64

In [79]:
df['is_Saint_Petersburg'].value_counts()

is_Saint_Petersburg
False    10495149
True       863001
Name: count, dtype: int64

In [80]:
df[df['is_Moscow'] & df['is_Saint_Petersburg']].shape[0] # 0 квартир и в Москве и в Санкт-Петербурге

0

In [81]:
df[df['is_Moscow'] | df['is_Saint_Petersburg']].shape[0] # 1'889'873 квартир в Москве и в Санкт-Петербург

1889873

In [82]:
df[df['is_Moscow'] | df['is_Saint_Petersburg']].shape[0] / df.shape[0] # 16% всех квартир

0.16638915668484744

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [83]:
df = df.drop(['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'], axis=1, errors='ignore')
df

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.299999,0.0,0,False,False
1,2021-01-01,1450000,5,5,1,33.000000,6.0,0,False,False
2,2021-01-01,10700000,4,13,3,85.000000,12.0,3,False,False
3,2021-01-01,3100000,3,5,3,82.000000,9.0,0,False,False
4,2021-01-01,2500000,2,3,1,30.000000,9.0,3,False,False
...,...,...,...,...,...,...,...,...,...,...
11358145,2021-12-31,6099000,4,9,3,65.000000,0.0,0,False,False
11358146,2021-12-31,2490000,1,10,2,56.900002,0.0,0,False,False
11358147,2021-12-31,850000,2,2,2,37.000000,5.0,0,False,False
11358148,2021-12-31,4360000,5,5,1,36.000000,9.0,0,False,False


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type, is_Moscow, is_Saint_Petersburg

Числовые: price, level, levels, rooms, area, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [84]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['building_type', 'is_Moscow', 'is_Saint_Petersburg']
encoder = OneHotEncoder(sparse_output=False, dtype=np.float32, handle_unknown='ignore') # создаем энкодер

encoded = encoder.fit_transform(df[categorical_features])
encoded_cols = encoder.get_feature_names_out(categorical_features) # генерим названия новых колонок

df_encoded = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)
df = pd.concat([df, df_encoded], axis=1)
df = df.drop(columns=categorical_features) # убираем старые признаки

print(df.head())

         date     price  level  levels  rooms       area  kitchen_area  \
0  2021-01-01   2451300     15      31      1  30.299999           0.0   
1  2021-01-01   1450000      5       5      1  33.000000           6.0   
2  2021-01-01  10700000      4      13      3  85.000000          12.0   
3  2021-01-01   3100000      3       5      3  82.000000           9.0   
4  2021-01-01   2500000      2       3      1  30.000000           9.0   

   building_type_0  building_type_1  building_type_2  building_type_3  \
0              1.0              0.0              0.0              0.0   
1              1.0              0.0              0.0              0.0   
2              0.0              0.0              0.0              1.0   
3              1.0              0.0              0.0              0.0   
4              0.0              0.0              0.0              1.0   

   building_type_4  building_type_5  building_type_6  is_Moscow_False  \
0              0.0              0.0        

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [85]:
import datetime

mn_date = datetime.date.fromisoformat(df['date'].min())
print(mn_date)
df['days_since_mn_date'] = df.apply( # это первый признак 
    lambda row: (datetime.date.fromisoformat(row['date']) - mn_date).days,
    axis=1,
).astype(np.int16)
df = df.drop('date', axis=1) # дропаем дату
df

2021-01-01


,price,level,levels,rooms,area,kitchen_area,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Petersburg_False,is_Saint_Petersburg_True,days_since_mn_date
0,2451300,15,31,1,30.299999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
1,1450000,5,5,1,33.000000,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
2,10700000,4,13,3,85.000000,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
3,3100000,3,5,3,82.000000,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,2500000,2,3,1,30.000000,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,6099000,4,9,3,65.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364
11358146,2490000,1,10,2,56.900002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364
11358147,850000,2,2,2,37.000000,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364
11358148,4360000,5,5,1,36.000000,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364


In [86]:
df['level_ratio'] = df['level'] / df['levels'].replace(0,1) # это второй признак, на всякий случай если levels == 0 заменяем 
df['level_ratio'] = df['level_ratio'].astype(np.float32) # чтобы поменьше весило
df

,price,level,levels,rooms,area,kitchen_area,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,is_Moscow_False,is_Moscow_True,is_Saint_Petersburg_False,is_Saint_Petersburg_True,days_since_mn_date,level_ratio
0,2451300,15,31,1,30.299999,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.483871
1,1450000,5,5,1,33.000000,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,1.000000
2,10700000,4,13,3,85.000000,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.307692
3,3100000,3,5,3,82.000000,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.600000
4,2500000,2,3,1,30.000000,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11358145,6099000,4,9,3,65.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364,0.444444
11358146,2490000,1,10,2,56.900002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364,0.100000
11358147,850000,2,2,2,37.000000,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364,1.000000
11358148,4360000,5,5,1,36.000000,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,364,1.000000


In [87]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerics = ['level','levels','rooms','area','kitchen_area','days_since_mn_date','level_ratio'] # выделили признаки которые нужно отнормировать
df = df.replace([np.inf, -np.inf], 0).fillna(0) # почистили датасет
df[[f"{col}_scaled" for col in numerics]] = scaler.fit_transform(df[numerics]) # добавим отнормированные признаки отдельными колонками

print(df.head()) # посмотрим что получилось

      price  level  levels  rooms       area  kitchen_area  building_type_0  \
0   2451300     15      31      1  30.299999           0.0              1.0   
1   1450000      5       5      1  33.000000           6.0              1.0   
2  10700000      4      13      3  85.000000          12.0              0.0   
3   3100000      3       5      3  82.000000           9.0              1.0   
4   2500000      2       3      1  30.000000           9.0              0.0   

   building_type_1  building_type_2  building_type_3  ...  \
0              0.0              0.0              0.0  ...   
1              0.0              0.0              0.0  ...   
2              0.0              0.0              1.0  ...   
3              0.0              0.0              0.0  ...   
4              0.0              0.0              1.0  ...   

   is_Saint_Petersburg_True  days_since_mn_date  level_ratio  level_scaled  \
0                       0.0                   0     0.483871      1.622769   
1 

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [88]:
from scipy.spatial.distance import cdist

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
  
    def fit(self, X, y): # установили выборку
        self.X_train = X
        self.y_train = y
  
    def predict(self, X):
        distances = cdist(X, self.X_train, metric=self.metric) # нашли дистанции
        neighbors_index = np.argsort(distances, axis=1)[:, :self.n_neighbors]  # взяли k ближайщих 
        neighbors_labels = self.y_train[neighbors_index] # взяли все выводы по ближайщим соседям
        y_pred = neighbors_labels.mean(axis=1) # взяли среднее из k ближайщих
        return y_pred # это считаем предсказанием


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [89]:
class LinearRegressionCustom:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate # все инициализировали
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
  
    def fit(self, X, y):
        m, n = X.shape # получаем размеры данных
        self.weights = np.zeros(n) # пока все веса в 0
        self.bias = 0.0

        if self.optimization == 'AdaGrad': # выбираем чем оптимизируем функцию потерь и в зависимости от этого заполняем вспомогательные параметры
            G = np.zeros(n) # для AdaGrad: сумма квадратов градиентов
        elif self.optimization == 'Momentum':
            v = np.zeros(n) # # для Momentum: накопленная скорость обновления
            beta = self.decay_rate # коэффициент сохранения момент
            
        for it in range(self.max_iter): # основной цикл обучения
            y_pred = X @ self.weights + self.bias # линейная комбинация X * weights + bias
            error = y_pred - y # вычисление ошибки 
            grad_w = (2/m) * X.T @ error # вычисление гардиента по весам
            grad_b = (2/m) * np.sum(error) # вычисление гардиента по смещению
            
            if self.optimization == 'SGD': # обновление весов 
                self.weights -= self.learning_rate * grad_w # корректируем веса
                self.bias -= self.learning_rate * grad_b 
            elif self.optimization == 'Momentum':
                v = beta * v + (1 - beta) * grad_w # экспоненциально взвешенное среднее градиентов
                self.weights -= self.learning_rate * v # корректируем веса
                self.bias -= self.learning_rate * grad_b 
            elif self.optimization == 'AdaGrad':
                G += grad_w**2 # накопление суммы квадратов градиентов 
                adjusted_lr = self.learning_rate / (np.sqrt(G) + self.epsilon) # каждый вес делится на корень из суммы квадратов своих градиентов
                self.weights -= adjusted_lr * grad_w # корректируем веса
                self.bias -= self.learning_rate * grad_b # смещение обновляем без адаптации
            else:
                raise ValueError('Unknown') # обработка ошибок
    
    def predict(self, X):
        return X @ self.weights + self.bias # делаем предсказание линейной модели


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

def print_metrics(model, X_train, y_train, X_test, y_test): # печатаем все ошибки 
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE: {mse:.2e}, MAE: {mean_absolute_error(y_test, y_pred):.2e}, RMSE: {np.sqrt(mse):.2e}")


exp_size = 80000 # возьмем только кусок выборки чтобы не ждать очень долго
df_exp = df.sample(exp_size, random_state=42)

features = [col for col in df_exp.columns 
            if ('_scaled' in col) or (col.startswith('building_type_')) or (col.startswith('is_'))] # отдельно возьмем все фичи

X = df_exp[features].values.astype(np.float32) # это все фичи в формате флот
y = df_exp['price'].values # это таргет - цена
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # разобьем на трейн тест

print('KNNRegressor из склерна')
skl_knn = KNeighborsRegressor(n_neighbors=5)
skl_knn.fit(X_train, y_train)
print_metrics(skl_knn, X_train, y_train, X_test, y_test)

print('LinearRegression из склерна')
skl_lr = LinearRegression()
skl_lr.fit(X_train, y_train)
print_metrics(skl_lr, X_train, y_train, X_test, y_test)

print('KNNRegressor свой')
custom_knn = KNNRegressor(n_neighbors=5)
custom_knn.fit(X_train, y_train)
print_metrics(custom_knn, X_train, y_train, X_test, y_test)

print('LinearRegression свой')
custom_lr = LinearRegressionCustom(learning_rate=0.01, max_iter=50)
custom_lr.fit(X_train, y_train)
print_metrics(custom_lr, X_train, y_train, X_test, y_test)


KNNRegressor из склерна
MSE: 9.00e+14, MAE: 2.92e+06, RMSE: 3.00e+07
LinearRegression из склерна
MSE: 5.07e+14, MAE: 4.32e+06, RMSE: 2.25e+07
KNNRegressor свой
MSE: 9.00e+14, MAE: 2.92e+06, RMSE: 3.00e+07
LinearRegression свой
MSE: 5.24e+14, MAE: 4.16e+06, RMSE: 2.29e+07


Видим, что самописная линейная реграссия показала себя чуть хуже библиотечной, а knn не отличается значимо